In [22]:
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

In [4]:
X_raw = loadmat("R2198_20ms.mat")
y_raw = np.loadtxt("R2198_locations.dat")

X_raw = X_raw['mm'].T

In [1]:
%run 'generate-dataset.ipynb'
# generate_datasets(window=50, step=25, test_set_sampling='chunk', test_set_size=0.2, test_chunk_location=0.8)

In [14]:
t = 20
for step_fraction in [1, 0.9, 0.75, 0.5, 0.25, 0.1]:
    print(int(t*step_fraction))

20
18
15
10
5
2


In [29]:
# results = []
models = [RandomForestRegressor(n_jobs=-1, verbose=0), 
            ExtraTreesRegressor(n_jobs=-1, verbose=0)]

best_mae = 20
# for time_window in [10,20,50,75]:
for time_window in [10,20,50,75, 100, 125, 150, 200]:

    print('window', time_window)
#     for step_fraction in [0.75, 0.5, 0.25]:
    for step_fraction in [1, 0.9, 0.1]:

        step = int(time_window*step_fraction)
        print('step', step_fraction)
#         for sampling in ['uniform_chunks', 'chunk']:
        X_train, X_test, y_train, y_test = generate_datasets(
                        window=time_window, step=step,  test_set_sampling='chunk')

        for clf in models:
            clf.fit(X_train, y_train)
            mae = mean_absolute_error(clf.predict(X_test), y_test)
            if mae < best_mae:
                best_mae = mae
            print(mae, best_mae)
            results.append({'window': time_window, 
                            'step': step,
                            'MAE': mae,
                            'model': clf.__class__.__name__})
    #             print(results)

window 10
step 1
generating data
window 10, step 10
sampling with a chunk, size 0.2, location 0.8
indexes 5132:6415
(1283, 660) (5132, 660) (1283, 2) (5132, 2)
16.9079567279 16.9079567279
16.9323576568 16.9079567279
step 0.9
generating data
window 10, step 9
sampling with a chunk, size 0.2, location 0.8
indexes 5701:7126
(1425, 660) (5702, 660) (1425, 2) (5702, 2)
17.0257172093 16.9079567279
16.6658415174 16.6658415174
step 0.1
generating data
window 10, step 1
sampling with a chunk, size 0.2, location 0.8
indexes 51320:64150
(12830, 660) (51320, 660) (12830, 2) (51320, 2)
16.4019532388 16.4019532388
15.9368863371 15.9368863371
window 20
step 1
generating data
window 20, step 20
sampling with a chunk, size 0.2, location 0.8
indexes 2565:3206
(641, 1320) (2566, 1320) (641, 2) (2566, 2)
17.0795959268 15.9368863371
16.1217296593 15.9368863371
step 0.9
generating data
window 20, step 18
sampling with a chunk, size 0.2, location 0.8
indexes 2850:3562
(712, 1320) (2851, 1320) (712, 2) (2851,

In [30]:
pickle.dump(results, open( "tree_results_chunk.p", "wb"))